## CS 454 TERM PROJECT - CAN SÖLÖMBAZ, MERT DEDEKÖY

In [6]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
import math
import itertools
from sklearn.model_selection import train_test_split

In [5]:
train=pd.read_csv("C:/Users/mrt/Desktop/ders/cs454/fashion-mnist_train.csv")
test=pd.read_csv("C:/Users/mrt/Desktop/ders/cs454/fashion-mnist_test.csv")

In [34]:
x_train=train.iloc[:,1:].values
r_train=train.iloc[:,0].values
x_test=test.iloc[:,1:].values
r_test=test.iloc[:,0].values

In [39]:
x_train, x_validation, r_train, r_validation = train_test_split(x_train, r_train, test_size=1/6, random_state=42)

## Multi Layer Perceptron

Since the problem is not linearly seperable, mlp should be used to perform a nonlinear regression

Network was designed as one output and one input and number of hidden units

I used batch gradient descent to find parameters

$ y^t = \sum_{h=1}^H v_hz_h^t+v_0$ 

$ z_h = sigmoid(w_h^tx) $

$ E(w,v|X) = 0.5\sum_{t=1}^N (r^t-y^t)^2 $

$\Delta v_h = \alpha \sum_{t=1}^N (r^t-y^t)z_h^t$

$ \Delta w_hj = \alpha \sum_{t=1}^N (r^t-y^t)v_h z_h^t (1-z_h^t) x_j^t $

In [ ]:
def loss_func():
    loss=0
    for data in range(50000):
        for Class in range(10):
            if r_train[data]==Class:
                loss=+ np.log(y[Class,data])
    loss=loss*-1
    return loss

In [ ]:
def mlp_params(alpha, number_of_H, x_train, r_train):

    w_all=[]
    v_all=[]
    for H in range (2,number_of_H+1):
        
        # initialize parameters
        x=np.ones((785,len(x_train)))
        x[1:,:]=np.transpose(x_train)
        v=np.random.uniform(-0.01,0.01,size=(10,H))#v_ih
        w=np.random.uniform(-0.01,0.01,size=(H-1,len(x))) #w_hj      
        z=1/(1+np.exp(-(w.dot(x))))
        o=np.zeros((10,50000))
        for i in range(50000):
            o[:,i]=v[:,1:].dot(z)[:,i]+v[:,0]
        y=np.exp(o)/(np.sum(np.exp(o),axis=0))
        y=np.argmax(y,axis=0)

    
        epoch=0
        
        while True:
        
            delta_v=np.empty(H)
            
            sum_v0=0
            for i in range(len(x_train)):
                    sum_v0+=(r_train[i]-y[i])
            delta_v[0]=alpha*sum_v0        
            for h in range(H-1):
                sum_1=0
                for i in range(len(x_train)):
                    sum_1+=(r_train[i]-y[i])*z[h,i]
                delta_v[h+1]=alpha*sum_1
            
            
            
            delta_w=np.empty((H-1,len(x)))
            
            for h in range(H-1):  
                for j in range(len(x)):
                    sum_2=0
                    for i in range(len(x_train)):
                        sum_2+=(r_train[i]-y[i])*v[1:][h]*z[h,i]*(1-z[h,i])*x[j,i]
                    delta_w[h,j]=alpha*sum_2
            
            
            temp_v=v
            v=np.add(v,delta_v,dtype=np.float64)
            
            
            temp_w=w      
            w=np.add(w,delta_w,dtype=np.float64)
            
    
            
            z=1/(1+np.exp(-(w.dot(x))))
            o=np.zeros((10,50000))
            for i in range(50000):
                o[:,i]=v[:,1:].dot(z)[:,i]+v[:,0]
            y=np.exp(o)/(np.sum(np.exp(o),axis=0))
            y=np.argmax(y,axis=0)

            
   
            epoch=epoch+1
            
            
            
            if (all(x <0.0001 for x in itertools.chain(*delta_w))) and (all(x <0.0001 for x in itertools.chain(*delta_v))):
                w_all.append(w)
                v_all.append(v)
                break
    
    return w_all,v_all

In [ ]:
w_all, v_all = mlp_params(0.01,10,x_train,r_train)

In [ ]:
def mlp_results(w_all, v_all, x_data, r_data, number_of_H):
    
    all_mse=[]
    y_all=[]
    for H in range (number_of_H-1):
        
        x=np.ones((2,len(x_data)))
        x[1,:]=x_data
        z=1/(1+np.exp(-(w_all[H].dot(x))))
        y=v_all[H][1:].dot(z)+v_all[H][0]
        
        y_all.append(y)
        all_mse.append(mean_square_error(y,r_data))
        
    return all_mse,y_all

In [ ]:
train_errors_mlp,y_train = mlp_results(w_all, v_all, x_train, r_train, 10)
validation_errors_mlp,y_validation = mlp_results(w_all, v_all, x_validation, r_validation, 10)

In [ ]:
def plt_error(validation_errors,train_errors,H):
    num_k=[k for k in range(2,H+1)] 
    plt.plot(num_k,validation_errors, label="validation error")
    plt.plot(num_k,train_errors, label="train error")
    plt.legend(loc="upper right")
    plt.xlabel("k")
    plt.ylabel("mean square error")
    plt.xticks(num_k)
    plt.show()
plt_error(validation_errors_mlp,train_errors_mlp,10)